In [1]:
%reload_ext autoreload
%autoreload 2

import os
from pathlib import Path
import multiprocessing

import jax
import jax.numpy as jnp
import pandas as pd
import numpy as np
import numpyro
import numpyro.distributions as dist

from hbmep.config import Config
from hbmep.model import Baseline
from hbmep.model.utils import Site as site
from hbmep.utils.constants import RECTIFIED_LOGISTIC

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()


#### Load config

In [2]:
root_path = Path(os.getcwd()).parent.parent.parent.absolute()
toml_path = os.path.join(root_path, "configs/J_RCML_000.toml")

config = Config(toml_path=toml_path)

config.BUILD_DIR = "/home/vishu/repos/hbmep-paper/reports/J_RCML_000/fix/inv"


2023-07-19 11:49:03,847 - hbmep.config - INFO - Verifying configuration ...
2023-07-19 11:49:03,847 - hbmep.config - INFO - Success!


In [3]:
class RectifiedLogistic(Baseline):
    def __init__(self, config: Config):
        super(RectifiedLogistic, self).__init__(config=config)
        self.link = RECTIFIED_LOGISTIC

    def _model(self, subject, features, intensity, response_obs=None):
        intensity = intensity.reshape(-1, 1)
        intensity = np.tile(intensity, (1, self.n_response))

        feature0 = features[0].reshape(-1,)

        n_data = intensity.shape[0]
        n_subject = np.unique(subject).shape[0]
        n_feature0 = np.unique(feature0).shape[0]

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate(site.n_subject, n_subject, dim=-2):
                """ Hyper-priors """
                mu_a = numpyro.sample(
                    site.mu_a,
                    dist.TruncatedNormal(150, 50, low=0)
                )
                sigma_a = numpyro.sample(site.sigma_a, dist.HalfCauchy(5))

                # sigma_b = numpyro.sample(site.sigma_b, dist.HalfNormal(0.5))

                # sigma_L = numpyro.sample(site.sigma_L, dist.HalfNormal(0.05))
                # sigma_H = numpyro.sample(site.sigma_H, dist.HalfNormal(5))
                # sigma_v = numpyro.sample(site.sigma_v, dist.HalfNormal(5))

                with numpyro.plate("n_feature0", n_feature0, dim=-3):
                    """ Priors """
                    a = numpyro.sample(
                        site.a,
                        dist.TruncatedNormal(mu_a, sigma_a, low=0)
                    )
                    b = numpyro.sample(site.b, dist.HalfCauchy(1))

                    L = numpyro.sample(site.L, dist.HalfCauchy(.05))
                    H = numpyro.sample(site.H, dist.HalfCauchy(2))
                    v = numpyro.sample(site.v, dist.HalfCauchy(5))

                    # p = numpyro.sample(site.p, dist.HalfCauchy(10))

                    sigma_1 = numpyro.sample(
                        "sigma_1", dist.HalfCauchy(10)
                    )
                    sigma_2 = numpyro.sample(
                        "sigma_2", dist.HalfCauchy(10)
                    )

                    d_1 = numpyro.sample(
                        "d_1", dist.HalfCauchy(10)
                    )
                    d_2 = numpyro.sample(
                        "d_2", dist.HalfCauchy(10)
                    )

        """ Model """
        mu = numpyro.deterministic(
            site.mu,
            L[feature0, subject]
            + jnp.maximum(
                0,
                -1
                + (H[feature0, subject] + 1)
                / jnp.power(
                    1
                    + (jnp.power(1 + H[feature0, subject], v[feature0, subject]) - 1)
                    * jnp.exp(-b[feature0, subject] * (intensity - a[feature0, subject])),
                    1 / v[feature0, subject]
                )
            )
        )
        sigma = numpyro.deterministic(
            "sigma",
            sigma_1[feature0, subject]
            + sigma_2[feature0, subject] * mu
        )
        d = numpyro.deterministic(
            "d",
            2 + d_1[feature0, subject]
            + d_2[feature0, subject] * mu
        )

        with numpyro.plate(site.data, n_data):
            return numpyro.sample(
                site.obs,
                dist.StudentT(df=d, loc=mu, scale=sigma).to_event(1),
                obs=response_obs
            )



model = RectifiedLogistic(config=config)

In [4]:
df = pd.read_csv(model.csv_path)
print(df.compound_position.unique())

['C5M-C5L' 'C5M-C6L' 'C7L-C8L' '-C5M' 'C7M-C8L' 'C8M-C8L' '-C8M' '-C6L'
 '-C7M' '-C8L' 'C7M-C8M' 'C5L-C6L' '-C5L' 'C6L-C7L' '-C7L' 'C7M-C7L'
 'C6M-C7M' 'C5M-C6M' '-C6M' 'C6M-C7L' 'C6M-C6L']


In [5]:


""" Filter """
# ind = df.compound_position.isin(["-C5M"])
sub = [
    ("amap01", "C5M-C5L"),
    ("amap01", "C5M-C6L"),
    ("amap01", "C7L-C8L"),
    # ("amap01", "-C5M"),
    # ("amap01", "C7M-C8L"),
    # ("amap01", "C8M-C8L"),
    # ("amap01", "-C8M"),
    # ("amap01", "-C6L"),
    # ("amap01", "-C7M"),
    # ("amap01", "-C8L"),
    # ("amap01", "C7M-C8M"),
    # ("amap01", "C5L-C6L"),
]
ind = df[model.combination_columns].apply(tuple, axis=1).isin(sub)
# ind = df[model.subject].isin(["amap01"])

df = df[ind].copy()
df.reset_index(drop=True, inplace=True)

df, encoder_dict = model.load(df=df)

df.shape

2023-07-19 11:49:04,548 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/J_RCML_000/fix/inv
2023-07-19 11:49:04,548 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/fix/inv
2023-07-19 11:49:04,549 - hbmep.dataset.core - INFO - Processing data ...
2023-07-19 11:49:04,550 - hbmep.utils.utils - INFO - func:load took: 0.00 sec


(173, 43)

In [6]:
mcmc, posterior_samples = model.run_inference(df=df)


2023-07-19 11:49:04,653 - hbmep.model.baseline - INFO - Running inference with rectified_logistic ...


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

2023-07-19 11:52:19,863 - hbmep.utils.utils - INFO - func:run_inference took: 3 min and 15.21 sec


In [7]:
mcmc.print_summary(prob=.95)



                    mean       std    median      2.5%     97.5%     n_eff     r_hat
      H[0,0,0]      0.77      0.08      0.75      0.63      0.92   4722.19      1.00
      H[0,0,1]      1.18     13.81      0.20      0.14      2.18   2194.07      1.00
      H[0,0,2]      8.64     58.90      2.57      0.44     26.35   6528.85      1.00
      H[1,0,0]      1.30      0.15      1.28      1.05      1.60   5447.55      1.00
      H[1,0,1]      1.03      5.95      0.54      0.37      1.54   2167.84      1.00
      H[1,0,2]      0.74      0.12      0.72      0.57      0.95   2712.22      1.00
      H[2,0,0]      0.72      0.07      0.71      0.60      0.87   4669.32      1.00
      H[2,0,1]      7.99     94.68      1.56      0.29     18.12   7409.89      1.00
      H[2,0,2]     19.02    643.03      2.09      0.17     26.12   5678.50      1.00
      L[0,0,0]      0.04      0.00      0.04      0.03      0.04   9286.99      1.00
      L[0,0,1]      0.02      0.00      0.02      0.01      0.02

In [8]:
model.render_recruitment_curves(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)


2023-07-19 11:52:20,173 - hbmep.model.baseline - INFO - Rendering recruitment curves ...
2023-07-19 11:52:32,621 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/fix/inv/recruitment_curves.pdf
2023-07-19 11:52:32,621 - hbmep.utils.utils - INFO - func:render_recruitment_curves took: 12.45 sec


In [9]:
model.render_predictive_check(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)


2023-07-19 11:52:32,668 - hbmep.model.baseline - INFO - Rendering Posterior Predictive Check ...


2023-07-19 11:52:47,319 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/fix/inv/posterior_predictive_check.pdf
2023-07-19 11:52:47,323 - hbmep.utils.utils - INFO - func:render_predictive_check took: 14.66 sec
